In [22]:
''' 
OSMNX Road Network Download Block 
'''

# Imports
import osmnx as ox
import geopandas as gpd
from shapely.geometry import LineString
import time

# year for file paths
year = 2018

# Set Timestamp
timestamp = str(year)+"-08-01T00:00:00Z"   

# OSMNX Overpass Settings
ox.settings.use_cache   = False
ox.settings.log_console = True
ox.settings.overpass_endpoint = "https://overpass-api.de/api/interpreter"   # Specifies default interpreter
ox.settings.overpass_settings = (f'[out:json][date:"{timestamp}"][timeout:18000]') # Load Timestamp

# Filter for road type
road_types = (
    '["highway"~"motorway|motorway_link|trunk|trunk_link|'
    'primary|primary_link|secondary|secondary_link"]')          
# This line ^^^ should be modified when downloading the 2024 base map, remove secondary/secondary link

# Load Graph from OSMNX
G = ox.graph_from_place(
    "Egypt", # Inbuilt Geographic tag
    network_type='all_private', #
    custom_filter=road_types,
    simplify=True, # Ensures proper topology
    retain_all=False) # Ignores disconnected fragments

# Ensure proper cache for next run
ox.settings.use_cache   = True

# OSMNX Automatically finds best meter projection
G = ox.project_graph(G)

# Creates Nodes GDF layer from OSMNX
nodes_gdf = ox.graph_to_gdfs(G, nodes=True, edges=False)

# Creates Edges GDF Layer from OSMNX
records = [] # List to store U,V Pairs
for u, v, key, data in G.edges(keys=True, data=True): # Keys and Data ensure proper attribute retrieval for each U,V pair
    geom = data.get('geometry',
                    LineString([
                      (G.nodes[u]['x'], G.nodes[u]['y']),
                      (G.nodes[v]['x'], G.nodes[v]['y'])
                    ]))
    rec = {
        'u': u,
        'v': v,
        'highway': data.get('highway'),
        'length': data.get('length'),
        'osmid': data.get('osmid'),
        'geometry': geom,
        'id': data.get('id'),
        'lanes': data.get('lanes'),
        'oneway': data.get('oneway')
    }
    records.append(rec)

edges_gdf = gpd.GeoDataFrame(records, crs=nodes_gdf.crs)

# Save to Geopackage
edges_gdf.to_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Egypt"+str(year)+"08_01_HW+PR+SE_OSMNX.gpkg", layer="edges", driver="GPKG")
nodes_gdf.to_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Egypt"+str(year)+"_08_01_HW+PR+SE_OSMNX.gpkg", layer="nodes", driver="GPKG")



C:\Users\Yan\Desktop\Python 3.10.6\lib\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 469 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


  → download took 18.08 min
✅ Written egypt_hwy_only.gpkg


In [12]:
""" 
Processing of NTL data Converting the NTL from Raster image to point geometries in ShapeFile 
"""

import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from rasterio.crs import CRS

# Enter Year of file being processed, does file saving automatically
year = 2024

# Input raster file path
input_path = "C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Nighttime Lights Raster Points/egypt_filtered_nighttime_lights_"+str(year)+".tif"
output_path = ("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Nighttime Lights Raster Points/Egypt_NTL_"+str(year)+"_Devectorized.shp")

# Using Rasterio to convert raster pixel values to points
with rasterio.open(input_path) as src:
    data = src.read(1)  # Assuming a single-band raster
    transform = src.transform  # Ensures matching coordinates
    no_data_value = src.nodata  # Fills in NaNs
    crs = CRS.from_epsg(32636) # Sets CRS
    
# Masks out all pixels with no data
if no_data_value is not None:
    valid_mask = (data != no_data_value) & np.isfinite(data) & (data > 0)
else:
    valid_mask = np.isfinite(data) & (data > 0)
    
rows, cols = np.where(valid_mask)  # Indices of valid pixels

# Convert row, col indices to geographic coordinates
x_coords, y_coords = rasterio.transform.xy(transform, rows, cols, offset="center")
pixel_values = data[rows, cols]  # Get pixel values directly using NumPy indexing

# Create geometries (points)
geometries = [Point(y, x) for y, x in zip(y_coords, x_coords,)]

# Convert to GDF
gdf = gpd.GeoDataFrame({"value": pixel_values}, geometry=geometries, crs=crs)

# Debug
print(gdf)

# Save
gdf.to_file(output_path, driver="ESRI Shapefile")



            value                 geometry
0        7.640000  POINT (3521250 -253250)
1        6.720833  POINT (3521250 -252750)
2        5.614286  POINT (3521250 -252250)
3       18.533333  POINT (3520750 -253250)
4       27.700000  POINT (3520750 -252750)
...           ...                      ...
125351  24.785714   POINT (2434250 309250)
125352   6.722222   POINT (2434250 309750)
125353   8.316667   POINT (2434250 346750)
125354   8.150000   POINT (2433750 346250)
125355  12.237500   POINT (2433750 346750)

[125356 rows x 2 columns]


In [34]:
'''
TAKES OSMNX DATA AND EXPLODES MULTILINESTRINGS AND ADDS ADDITIONAL VERTICES EVERY 100M
'''

import numpy as np
from shapely.geometry import Point
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString

# Year Variable for file paths
year = 2023

gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_06_01_QGIS_Clipped.gpkg", layer="edges")
output_path = ("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_06_01_Step1_interpolated.gpkg")

# Set CRS
gdf = gdf.set_crs("EPSG:32636", allow_override=True)
print(gdf)
# Drop Unnecessary Columns
gdf_columns = gdf.columns
gdf_necessary_columns = ['osmid','lanes', 'geometry', 'highway','oneway']

for x in gdf_columns:
    if x in gdf_necessary_columns:
        pass
    else: 
        gdf = gdf.drop(columns = [x])

# Turns Oneway info to integers
gdf["oneway"] = gdf["oneway"].map({
    "yes": 1,
    "-1": -1,
    "no": 0
}).fillna(0)


# Splits multiLinestrings into individual linestrings
def split_multilinestring(row, id_column, start_id):
    #Empty List
    geometries = []

    #Grab Row info to preserve attributes
    base_id = row[id_column]
    geometry = row['geometry']

    
    if geometry.geom_type == 'MultiLineString':
        original_start = geometry.geoms[0].coords[0] #Get first and last point of entire multilinestring 
        original_end = geometry.geoms[-1].coords[-1]
        
        for i, line in enumerate(geometry.geoms): # In case a line is split, a new, unique ID is given
            new_row = row.copy()
            new_row[id_column] = f"{base_id}_{i + start_id}"

            # Check direction of sub-line
            sub_start, sub_end = line.coords[0], line.coords[-1]
            dist_to_start = Point(sub_start).distance(Point(original_start))
            dist_to_end = Point(sub_end).distance(Point(original_start))

            if dist_to_end < dist_to_start:
                line = LineString(list(line.coords)[::-1])  # reverse if misaligned
            
            # Add subline to main geometries list
            new_row['geometry'] = line
            geometries.append(new_row)

    #Add Linestring to empty list
    elif geometry.geom_type == 'LineString':
        geometries.append(row)  # direction is preserved

    return geometries


# Reassemble gdf with split linestrings
id_column = 'osmid'  # Replace with the actual column name for IDs
start_id = 1  # Starting integer for the extra ID
split_rows = []
for _, row in gdf.iterrows():
   split_rows.extend(split_multilinestring(row, id_column, start_id))
gdf = gpd.GeoDataFrame(split_rows, crs=gdf.crs)



def interpolate_line_with_nodes(line, distance):
    """
    Adds points along a LineString every `distance` meters.
    Keeps original points and adds interpolated ones.
    """
    # Extract existing points
    existing_points = list(line.coords)

    # Generate interpolated points
    interpolated_points = [line.interpolate(d).coords[0] for d in np.arange(0, line.length, distance)]
    interpolated_points.append(line.interpolate(line.length).coords[0])  # Ensure the endpoint is included

    # Combine original and interpolated points
    all_points = sorted(existing_points + interpolated_points, key=lambda p: line.project(Point(p)))

    # Ensure no duplicates in the final LineString
    all_points = list(dict.fromkeys(all_points))  # Remove duplicates while preserving order

    # Print debugging information
    #print(f"Line segment length: {line.length:.2f} meters")
    #print(f"Points before: {len(existing_points)}, Points after: {len(all_points)}")
    #print("-" * 50)

    return LineString(all_points)

# Iterate over dataframe by line, apply interpolate with nodes. Important to note that since CRS is in 32636, 100 is in meters
gdf['geometry'] = gdf['geometry'].apply(lambda line: interpolate_line_with_nodes(line, 100))
#print(gdf)

# Create a new GeoDataFrame
new_gdf = gpd.GeoDataFrame(gdf, crs=gdf.crs)

# Export
new_gdf.to_file(output_path, layer="edges", driver="GPKG")

                 u            v     highway      length  \
0         38566657   6303863521     primary  441.699889   
1         38566657   7843867591     primary  133.532631   
2       6303863521   6040298543     primary   47.307494   
3       6303863521     38566657     primary  441.699889   
4       7843867591   6303863546     primary  158.056940   
...            ...          ...         ...         ...   
46932   7826432526  10704280124       trunk   92.116208   
46933  11005447178  11005447179  trunk_link   23.190743   
46934  11005586851  11005586884       trunk   49.608574   
46935  11005586884  11005586881  trunk_link   10.141817   
46936  11005586884   1167516307       trunk  162.931446   

                                      osmid    id lanes  oneway  \
0                                 673142480  None  None   False   
1                                 673142476  None  None    True   
2                                 673142474  None  None    True   
3                      

In [35]:
'''
TAKES NTL POINTS AND EXPLODED OSMNX MAP AND ADDS NORMALIZED PRIORITY SCORES TO ALL THE POINTS
'''

import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from scipy.spatial import cKDTree

# Year Variable for file paths
#year = 2022

# Load the shapefiles
ntlpoints_gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+
                              "/Nighttime Lights Raster Points/Egypt_NTL_"+str(year)+"_Devectorized.shp")
roads_gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+
                          "/Road Network/Egypt_"+str(year)+"_06_01_Step1_interpolated.gpkg", layer="edges")

# Output paths
output_path = ("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+
               "/Road Network/Egypt_"+str(year)+"_Step2_NTL_priorities_added.gpkg")

# These should be set to 32636 to begin with but double check 
ntlpoints_gdf = ntlpoints_gdf.to_crs(epsg=32636)
roads_gdf = roads_gdf.to_crs(epsg=32636)

# Extract list of unique nodes for comparison with NTL points
unique_nodes = {}
for idx, row in roads_gdf.iterrows():
    line_id = row['osmid'] #osmid is osmnx provided id column
    geometry = row['geometry']
    
    if geometry.geom_type == 'LineString':
        for coord in geometry.coords:
            if coord not in unique_nodes: #prevents duplicates
                unique_nodes[coord] = {'osmid': line_id, 'coords': coord, 'geometry': Point(coord)} #append

# Create a GeoDataFrame for unique linestring nodes
nodes_list = list(unique_nodes.values())
nodes_gdf = gpd.GeoDataFrame(nodes_list, geometry='geometry', crs=roads_gdf.crs)

#set default prio_score to low number but not 0, prevent possible divide by 0
nodes_gdf['prio_score'] = 1e-10

#cKDTree allows quick search via spatial index
node_coords = np.array(list(unique_nodes.keys()))
tree = cKDTree(node_coords)

#Build list of the road network nodes closest to each NTL point, and the distance to it
distances = []            # To store distances
closest_node_coords = []  # To store closest node coordinates
closest_node_ids = []     # To store closest node IDs

#Filter out invalid points, zeroes and infinity
ntlpoints_gdf["geometry"] = ntlpoints_gdf["geometry"].apply(lambda g: Point(0, 0) if not g or not np.isfinite(g.x) or not np.isfinite(g.y) else g)

#Find the nearest node and store information about it
for point in ntlpoints_gdf.geometry: # Loop over all NTL points
    dist, idx = tree.query((point.x, point.y)) #use KDTree to find nearest node. Tree is built from road network nodes
    distances.append(dist)
    nearest_coord = tuple(node_coords[idx])
    closest_node_coords.append(nearest_coord)
    closest_node_ids.append(unique_nodes[nearest_coord]['osmid'])

### This section calculatutes NTL Priority Scores

# Add the results as attributes to the points GeoDataFrame
ntlpoints_gdf['distance_to_node'] = distances
ntlpoints_gdf['closest_node_coords'] = closest_node_coords
ntlpoints_gdf['closest_line_id'] = closest_node_ids

# Needed variables for normalization
max_priority = ntlpoints_gdf['value'].max()
max_distance = ntlpoints_gdf['distance_to_node'].max()


# Normalizes the light intensity value of the NTL pixel
ntlpoints_gdf['normalized_priority'] = ntlpoints_gdf['value'] / max_priority

# Normalizes the "Distance factor" of each NTl pixel
ntlpoints_gdf['normalized_distance'] = ntlpoints_gdf['distance_to_node'] / max_distance

# Computes prio_score by dividing normalized priority over normalized distance, plus decimal for DB0
ntlpoints_gdf['prio_score'] = ntlpoints_gdf['normalized_priority'] / ntlpoints_gdf['normalized_distance'] + 1e-11

### This section takes the NTL Priority scores and adds them to the nearest nodes on the road network

# Create a mapping from node coordinates (tuple) to index in nodes_gdf.
node_index_map = {tuple(geom.coords[0]): idx for idx, geom in enumerate(nodes_gdf['geometry'])}


# Iterate over the points and update the nearest node's priority score
for _, point_row in ntlpoints_gdf.iterrows():
    point_priority = point_row['prio_score']
    nearest_node_coords = point_row['closest_node_coords']  #Store as values for analysis
    # Ensure nearest_node_coords is a tuple.
    if isinstance(nearest_node_coords, np.ndarray):
        nearest_node_coords = tuple(nearest_node_coords)
    if nearest_node_coords in node_index_map:
        node_idx = node_index_map[nearest_node_coords]
        nodes_gdf.at[node_idx, 'prio_score'] += point_priority

# Create a mapping of node coordinates to priority scores.
node_prio_score_map = nodes_gdf.set_index(
    nodes_gdf['geometry'].apply(lambda geom: tuple(geom.coords[0]))
)['prio_score'].to_dict()

# Create a new GeoDataFrame for nodes using the mapping.
node_coords = list(node_prio_score_map.keys())
prio_scores = list(node_prio_score_map.values())

nodes_layer = gpd.GeoDataFrame(
    {"prio_score": prio_scores},
    geometry=[Point(coord) for coord in node_coords],
    crs=nodes_gdf.crs
)

#Create backup id index
nodes_layer["id"] = nodes_layer.index.astype(str)

#debug
#print(roads_gdf)

# Save the updated datasets, including the new nodes layer. nodes will be the vertices on the roads, while we save NTL points just for future reference, not mandatory
ntlpoints_gdf.to_file(output_path, layer="points", driver="GPKG")
roads_gdf.to_file(output_path, layer="edges", driver="GPKG")
nodes_layer.to_file(output_path, layer="nodes", driver="GPKG")

In [2]:
'''
ASSIGNS MISSING LANE NUMBERS TO EDGES SO THAT ROAD PRIO SCORE CAN BE CALCULATED
'''

import geopandas as gpd
import pandas as pd

# Year for file paths
#year = 2022

# Load the edges layer
edges_gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_Step2_NTL_priorities_added.gpkg", layer='edges')
nodes_gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_Step2_NTL_priorities_added.gpkg", layer='nodes')
output_path = "C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_Step3_Lane_Adjusted.gpkg"
print(edges_gdf)

# Convert "lanes" to integer, handle NaN
edges_gdf['lanes'] = pd.to_numeric(edges_gdf['lanes'], errors='coerce')  # Convert non-numeric to NaN

# For highways without marked lane numbers, we are using the average of all roads with lanes.
avg_lanes_per_highway = edges_gdf.groupby('highway')['lanes'].mean()

# Define a function to fill missing lane values with average based on highway type
def fill_missing_lanes(row):
    if pd.isna(row['lanes']):  # If there is no existing lane count, then apply mean
        return avg_lanes_per_highway.get(row['highway'], 1)  # Default to 1 lane if no data for that highway type
    return row['lanes']

# Apply function to fill missing lane values
edges_gdf['lanes'] = edges_gdf.apply(fill_missing_lanes, axis=1)

# debug
#print("Original missing lane values:", edges_gdf['lanes'].isna().sum())
#print("Missing lane values after filling:", edges_gdf['lanes'].isna().sum())

# Save the updated dataset back to the GeoPackage

nodes_gdf.to_file(output_path, layer="nodes", driver="GPKG")
edges_gdf.to_file(output_path, layer="edges", driver="GPKG")
# Debug
print(edges_gdf)


       highway        osmid     lanes  oneway  \
0      primary    5376791_1  2.633406     0.0   
1      primary    5376791_1  2.633406     0.0   
2      primary    5376791_1  2.633406     0.0   
3      primary    5376791_1  2.633406     0.0   
4      primary    5376791_1  2.633406     0.0   
...        ...          ...       ...     ...   
16301    trunk  101123308_1  2.226917     0.0   
16302    trunk  101123223_1  2.226917     0.0   
16303    trunk  101123237_1  2.226917     0.0   
16304    trunk  101123285_1  2.226917     0.0   
16305    trunk  101123223_1  2.226917     0.0   

                                                geometry  
0      LINESTRING (488036.749 2659087.016, 488098.336...  
1      LINESTRING (488036.749 2659087.016, 487945.482...  
2      LINESTRING (488098.336 2659089.422, 488036.749...  
3      LINESTRING (488098.336 2659089.422, 488163.269...  
4      LINESTRING (487842.988 2659092.239, 487881.371...  
...                                                  ... 

In [42]:
''' Creates Proper Road Network by splitting the
    road network along each point with high prior score, 
    intersection points, Assigns Priority scores to Roads '''
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.ops import split
from shapely.geometry import MultiPoint, Point, LineString
from shapely.strtree import STRtree
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
from shapely.ops import unary_union
from shapely.geometry import Point
from shapely.geometry import LineString
import uuid


# Year for file paths
#year = 2022

# Load file
edges_gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_Step3_Lane_Adjusted.gpkg", layer='edges')
nodes_gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_Step3_Lane_Adjusted.gpkg", layer='nodes')
output_path = "C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_Step4_MSPL_Ready.gpkg"
print(edges_gdf)

# Check for proper CRS, most likely redundant but good to check
edges_gdf = edges_gdf.to_crs(epsg=32636)
nodes_gdf = nodes_gdf.to_crs(edges_gdf.crs)

### The first step in this code block is to isolate all dead-end points, all intersection points and all points on a line
### with priority scores over zero, and break up linestrings along them to each point can be assessed in a proper network

# Filter for nodes with non-zero priority scores
prio_nodes_gdf = nodes_gdf[nodes_gdf['prio_score'] > 1e-10]

# First, we will find all dead end points
endpoints = []

# Get every start and end point in geom
for geom in edges_gdf.geometry:
    if isinstance(geom, LineString):
        coords = list(geom.coords)
        endpoints.append(coords[0])
        endpoints.append(coords[-1])


# Count number of appearances for each endpoint
point_counts = Counter(endpoints)

# If it shows only once, add the coordinates to the set
deadend_coords = [Point(pt) for pt, count in point_counts.items() if count == 1]
deadend_set = set((pt.x, pt.y) for pt in deadend_coords)

# Filter deadend nodes out of gdf
deadend_gdf = nodes_gdf[nodes_gdf.geometry.apply(lambda pt: (pt.x, pt.y) in deadend_set)]

# Next, we find all the intersection nodes

# Extract all coordinates from all LineStrings
all_coords = []

#Loop over edges to add them nodes in
for geom in edges_gdf.geometry:
    if isinstance(geom, LineString):
        all_coords.extend(list(geom.coords))

# Count how many times each point appears
point_counts = Counter(all_coords)

# Intersections are points that appear more than twice
intersection_coords = [Point(pt) for pt, count in point_counts.items() if count > 2]

# Grab the coords of the intersection as tuples
intersection_set = set((pt.x, pt.y) for pt in intersection_coords)

# Filter nodes_gdf to include only intersection points
intersection_gdf = nodes_gdf[nodes_gdf.geometry.apply(lambda pt: (pt.x, pt.y) in intersection_set)]

print('intersections:', len(intersection_gdf))
print('prios', len(prio_nodes_gdf))
print('deadends', len(deadend_gdf))

#Create one GDF with all three classes we want to split linestrings by
combined_gdf = gpd.GeoDataFrame(
    pd.concat([prio_nodes_gdf, deadend_gdf, intersection_gdf], ignore_index=True),
    crs=prio_nodes_gdf.crs)

# Drop duplicates based on geometry only
combined_gdf = combined_gdf.drop_duplicates(subset='geometry')

# Debug
#print(combined_gdf)

# Build a Spatial Index for Fast Edge Lookup
spatial_index = STRtree(edges_gdf.geometry)

# Takes the points in the combined_gdf, and gets the rows of the edges nearest to them, allowing them to be split
multi_point = MultiPoint([pt for pt in combined_gdf.geometry])
lines_to_split_idx = spatial_index.query(multi_point)  # Get indices of relevant edges

relevant_edges = edges_gdf.iloc[lines_to_split_idx]  # Extract only the edges near candidate nodes

# To split linestrings quickly, we will build a numpy array of points we want to split by
node_coords = np.array([[pt.x, pt.y] for pt in combined_gdf.geometry])

def split_line_optimized(line):
    """
    Splits a LineString at candidate node points, preserving the original geometry's directionality.
    """
    line_coords = np.array(line.coords)
    distances = np.linalg.norm(node_coords[:, None] - line_coords, axis=2) # Calculates distance between the node point and vertices
    nearby_points = node_coords[np.any(distances < 1e-5, axis=1)] # covers for possible topological mismatches

    if len(nearby_points) == 0: #If not near any points, just return it unchanged
        return [line]

    # Perform split with shapely, splitting line at the points found by nearby_points
    split_result = split(line, MultiPoint([Point(p) for p in nearby_points]))
    valid_segments = [seg for seg in split_result.geoms if len(seg.coords) > 1] # Removes segments without 2 coords

    if not valid_segments:
        return [line]

    # Sort by projected distance along the original line (preserves direction)
    valid_segments = sorted(
        valid_segments,
        key=lambda seg: line.project(Point(seg.coords[0])))

    return valid_segments

# Apply the optimized split function to each relevant edge.
split_segments = [
    {"geometry": segment, **row.to_dict()}  # Store original attributes with each segment
    for _, row in relevant_edges.iterrows() # Applies this only to edges near points of interest
    for segment in split_line_optimized(row.geometry)] #Run functions

split_edges_gdf = gpd.GeoDataFrame(split_segments, crs=relevant_edges.crs) # Compile results into new GDF

#Debug print statements
#print("Split Edges GDF:", split_edges_gdf)
#print("Edges GDF:", edges_gdf)
#print("Combined GDF:", combined_gdf)

### Here, priority scores for edges will be calculated

#Calculate the length (in meters) for each edge.
split_edges_gdf["length_m"] = split_edges_gdf.geometry.length
split_edges_gdf['lanes'] = pd.to_numeric(split_edges_gdf['lanes'], errors='coerce')

# Calculate maximum values for normalization.
max_length = split_edges_gdf['length_m'].max()
max_lanes  = split_edges_gdf['lanes'].max()

# Normalize attributes and compute a road priority score capacity/length. This way, higher lanes correlates with a higher score, and longer roads with a lower score. 
split_edges_gdf['normalized_length'] = split_edges_gdf['length_m'] / max_length # Normalized Length
split_edges_gdf['normalized_capacity'] = split_edges_gdf['lanes'] / max_lanes #Normalized Lanecount "Capacity"
split_edges_gdf["road_prio_score"] = split_edges_gdf["normalized_capacity"] / (split_edges_gdf["normalized_length"] + 1e-6)

# Extract start and end coords from each LineString
endpoints = set()

#store start and end points separately
for geom in split_edges_gdf.geometry:
    if geom and geom.geom_type == 'LineString':
        coords = list(geom.coords)
        endpoints.add(coords[0])  
        endpoints.add(coords[-1])  

# Convert to list of Point geometries
unique_endpoints = [Point(xy) for xy in endpoints]

# Convert unique endpoint Points to coordinate tuples
endpoint_set = set((pt.x, pt.y) for pt in unique_endpoints)

# Filter nodes_gdf by checking if each point matches a known endpoint
endpoint_gdf = nodes_gdf[nodes_gdf.geometry.apply(lambda pt: (pt.x, pt.y) in endpoint_set)]

# Save the edges and nodes layers into the gpkg.

split_edges_gdf.to_file(output_path, layer="edges", driver="GPKG")
endpoint_gdf.to_file(output_path, layer="nodes", driver="GPKG")


          highway                                   osmid     lanes  oneway  \
0         primary                             673142480_1  2.570527     0.0   
1         primary                             673142476_1  2.570527     0.0   
2         primary                             673142474_1  2.570527     0.0   
3         primary                             673142480_1  2.570527     0.0   
4         primary                             673142476_1  2.570527     0.0   
...           ...                                     ...       ...     ...   
47339       trunk  [1184793549, 1184793550, 1184793551]_1  2.462956     0.0   
47340  trunk_link                            1184793556_1  1.725490     0.0   
47341       trunk                            1150717111_1  2.462956     0.0   
47342  trunk_link                            1184810310_1  1.725490     0.0   
47343       trunk                            1150717111_1  2.462956     0.0   

                                                geo

In [3]:
"""TQDM VERSION + Sampling + Working version"""
import geopandas as gpd
import igraph as ig
from shapely.geometry import Point
import numpy as np
import pandas as pd
import igraph as ig
from itertools import combinations
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib
from joblib import Parallel, delayed

#Year variable
year = 2014

#Debug, prints file path
#edge_path = "C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_analysis_ready_OG.gpkg"
#print(edge_path)

# Load GeoDataFrames
edges_gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_analysis_ready_OG.gpkg", layer="edges")
nodes_gdf = gpd.read_file("C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/"+str(year)+"/Road Network/Egypt_"+str(year)+"_analysis_ready_OG.gpkg", layer="nodes")
edges_gdf = edges_gdf.rename(columns={"osmid":"id"})

# Ensure all geometries use the same CRS
edges_gdf = edges_gdf.to_crs(nodes_gdf.crs)
edges_gdf_cleaned = edges_gdf.drop_duplicates()
edges_gdf = edges_gdf_cleaned.loc[~edges_gdf_cleaned.duplicated(keep="first")].reset_index(drop=True)

# Extract start and end points from each LineString
edges_gdf["start_geom"] = edges_gdf.geometry.apply(lambda geom: Point(geom.coords[0]))
edges_gdf["end_geom"] = edges_gdf.geometry.apply(lambda geom: Point(geom.coords[-1]))

# Convert start/end to GeoDataFrames
start_points = gpd.GeoDataFrame(edges_gdf[["id"]], geometry=edges_gdf["start_geom"], crs=edges_gdf.crs)
end_points = gpd.GeoDataFrame(edges_gdf[["id"]], geometry=edges_gdf["end_geom"], crs=edges_gdf.crs)

# Spatial join to find nearest node for each start and end point
start_join = gpd.sjoin_nearest(start_points, nodes_gdf[["id", "geometry"]], how="left", distance_col="dist")
end_join = gpd.sjoin_nearest(end_points, nodes_gdf[["id", "geometry"]], how="left", distance_col="dist")

# Merge results back into edges_gdf
edges_gdf["start_node_id"] = start_join["id_right"]
edges_gdf["end_node_id"] = end_join["id_right"]

# Standardize node IDs for igraph
nodes_gdf["id"] = nodes_gdf["id"].astype(str)
edges_gdf["start_node_id"] = edges_gdf["start_node_id"].astype(str)
edges_gdf["end_node_id"] = edges_gdf["end_node_id"].astype(str)

valid_ids = set(nodes_gdf["id"])
edges_gdf = edges_gdf[
    edges_gdf["start_node_id"].isin(valid_ids) &
    edges_gdf["end_node_id"].isin(valid_ids)
]
# Build edge list for igraph
edge_list = []
edge_ids = []
weights = []
edge_lengths = []

# start storing edges and attributes needed from the gdf in above lists
for _, row in edges_gdf.iterrows():
    sid = row["start_node_id"]
    eid = row["end_node_id"]
    edge_id = str(row["id"])
    weight = 1 / row["road_prio_score"] if row["road_prio_score"] > 0 else np.inf
    length  = row["length_m"]
    if row["oneway"] == 1:
        edge_list.append((sid, eid))
        edge_ids.append(edge_id)
        weights.append(weight)
        edge_lengths.append(length)

    elif row["oneway"] == -1:
        edge_list.append((eid, sid))
        edge_ids.append(edge_id)
        weights.append(weight)
        edge_lengths.append(length)
    elif row["oneway"] == 0:
        edge_list.append((eid, sid))
        edge_ids.append(edge_id)
        weights.append(weight)
        edge_lengths.append(length)

"""UNHASHTAG If USING ONE WAY"""
    #elif row["oneway"] == 0:
        #for u, v in [(sid, eid), (eid, sid)]:
            #edge_list.append((u, v))
            #edge_ids.append(edge_id)
            #weights.append(weight)

# Create new sets in case any duplicates were stored
edge_set = set()
clean_edges = []
clean_ids = []
clean_weights = []
clean_lengths = []

for (u,v), eid, w, L in zip(edge_list, edge_ids, weights, edge_lengths):
    if (u,v) not in edge_set:
        edge_set.add((u,v))
        clean_edges.append((u, v))
        clean_ids.append(eid)
        clean_weights.append(w)
        clean_lengths.append(L)

# Build the graph
g = ig.Graph(directed=True)
g.add_vertices(nodes_gdf["id"].tolist())
g.add_edges(clean_edges)
g.es["edge_id"] = clean_ids
g.es["weight"] = clean_weights
g.es["length"]  = clean_lengths  
g.vs["prio_score"] = nodes_gdf.set_index("id").loc[g.vs["name"]]["prio_score"].tolist()
print(f"✅ Final graph has {g.vcount()} nodes and {g.ecount()} edges.")

# Extract largest strongly connected component (LSCC) Redundant but just in case
largest_cc = g.clusters(mode="WEAK").giant()
print("LWCC edges:", largest_cc.ecount())
print("LWCC nodes:", largest_cc.vcount())

#Set parameters for the graph and analysis
full_edges = largest_cc.get_edgelist()
lengths    = largest_cc.es["length"]
vs       = largest_cc.vs
prios    = np.array(vs["prio_score"], dtype=float) #Will use priority scores for node subsampling
probs    = prios / prios.sum() #Set probabilities for subsampling
k        = min(15000, len(vs)) #Set sample size

#Do Sampling using random choice 
sample_idx = np.random.choice(len(vs), size=k, replace=False, p=probs)

#Function to do subsampling
def weighted_mspl_sampled(graph, sample_idx):
    # get prio weights for the sampled nodes
    pr = np.array(graph.vs["prio_score"], dtype=float)[sample_idx]
    # compute pairwise shortest‐path distances *only* between those sampled nodes
    D  = np.array(graph.shortest_paths_dijkstra(
            source=sample_idx,
            target=sample_idx,
            weights="weight"
         ))
    # build the outer‐product weight matrix
    P  = pr[:,None] * pr[None,:]
    # mask upper triangle, finite distances
    mask = np.triu(np.isfinite(D), k=1)
    return (P[mask] * D[mask]).sum() / P[mask].sum()

# 4) Compute your “baseline” on the sample
baseline_mspl = weighted_mspl_sampled(largest_cc, sample_idx)
print("Baseline (sampled) MSPL:", baseline_mspl)

#Filter edges by length for processing
full_edges  = largest_cc.get_edgelist()
lengths     = largest_cc.es["length"]
edge_list   = [(u,v) for (u,v),L in zip(full_edges, lengths) if L > 500] # Only takes edges over 500m, avoid running calculations over tiny segments
orig_ids    = [eid for eid,L in zip(largest_cc.es["edge_id"], lengths) if L > 500] 
orig_wts    = [w   for w, L in zip(largest_cc.es["weight"],    lengths) if L > 500]

#Set up calculations for multiprocessing
tasks = list(zip(edge_list, orig_ids, orig_wts))

# Define function that finds delta_MSPL in a way compatible with multicore processing
def process_edge(task):
    (u, v), eid, w = task
    # find igraph‐internal id and stash original weight
    i_eid     = largest_cc.get_eid(u, v)
    old_weight = largest_cc.es[i_eid]["weight"]

    # “remove” the edge by setting weight = ∞
    largest_cc.es[i_eid]["weight"] = float("inf")

    # recompute sampled MSPL
    mspl2 = weighted_mspl_sampled(largest_cc, sample_idx)
    delta = mspl2 - baseline_mspl

    # restore original weight
    largest_cc.es[i_eid]["weight"] = old_weight

    return {
        "id":  eid,
        "weight":            w,
        "mspl_after_removal": mspl2,
        "delta_mspl":         delta
    }

# Run "process_edge" in parallel on tqdm, n_jobs will correspond to core count
with tqdm_joblib(tqdm(desc="Processing edges", total=len(tasks))) as progress_bar:
    results = Parallel(n_jobs=4, backend="loky")(
        delayed(process_edge)(t) for t in tasks
    )

# Collect results
df = pd.DataFrame(results)

# … then your filtering & printing as before …
filtered_df = df[
    df["delta_mspl"].apply(np.isfinite) &
    (df["delta_mspl"] != 0)
]

results_gdf = edges_gdf.merge(filtered_df, on="id", how="left")
results_gdf = results_gdf.drop(['start_geom','end_geom'], axis=1)

# Save Result to File
results_gdf.to_file('C:/Users/Yan/Desktop/McCord Project Infra/Shapefiles/Egypt/'+str(year)+'/Road Network/Egypt_'+str(year)+'_MSPL_RESULTS_HighParam15000_run1.gpkg', layer="edges", driver="GPKG")

###Debug
#print("Top 10 Positive delta_mspl:")
#print(filtered_df.nlargest(10, "delta_mspl")[["id","delta_mspl"]])

#print("Top 10 Negative delta_mspl:")
#print(filtered_df.nsmallest(10, "delta_mspl")[["id","delta_mspl"]])


KeyboardInterrupt

